<a href="https://colab.research.google.com/github/jiya2107/Machine-Learning---Practical/blob/main/ML_Experiment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Load and preprocess data (same as experiment 3)
try:
    df = pd.read_csv('HepatitisCdata.csv', index_col=0)
except FileNotFoundError:
    print("Please ensure 'HepatitisCdata.csv' is uploaded.")
    exit()

df = df.replace('?', np.nan)
for col in ['ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT']:
    df[col] = pd.to_numeric(df[col])
df = df.dropna()

df['Sex'] = df['Sex'].map({'m': 1, 'f': 0})
# Handle '0s=suspect Blood Donor' by replacing '0s' with '0' before splitting
df['Category'] = df['Category'].str.replace('0s=', '0=').str.split('=').str[0].astype(int)

X = df.drop('Category', axis=1)
y = df['Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Logistic Regression Implementation ---
# Using solver='liblinear' for small datasets and 'multi_class='auto' for multi-class
log_reg = LogisticRegression(solver='liblinear', multi_class='auto', random_state=42)
log_reg.fit(X_train_scaled, y_train)

# Predict
y_pred = log_reg.predict(X_test_scaled)
y_pred_proba = log_reg.predict_proba(X_test_scaled)

# Evaluate
print("--- Logistic Regression Results ---")
print(f"Model Intercept (Bias): {log_reg.intercept_}")
# Note: Coeficients are multi-dimensional for multi-class classification
print(f"Model Coefficients (Weights): {log_reg.coef_[:1]}")

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print(f"\nSample Predicted Probability for first 5 test samples:\n{y_pred_proba[:5].round(4)}")